In [1]:
import numpy as np
import h5py
import trimesh
import matplotlib.pyplot as plt
from random import random, uniform
import dlib
from scipy.optimize import lsq_linear

In [13]:
%run features/basel_features/index_loader.ipynb
%run visualizers/model_mesh.ipynb
%run transformations/transformations.ipynb
%run basel_model/BaselModel.ipynb
%run optimizers/GradientDescent.ipynb
%run optimizers/initializers/EstimationRefinementInitializer.ipynb
%run optimizers/inputs/SingleImageInput.ipynb
%run optimizers/outputs/FaceModelMeshViewer.ipynb

In [15]:

baselModel = BaselModel('./model2017-1_bfm_nomouth.h5')
        
# get indexes of model feature points and associated dlib feature point numbers, indexed at 1
feature_point_indexes, dlib_points = get_feature_point_indexes('./feature_points.csv')
        
# extract only the feature points
shape_mean_features = baselModel.getShapeMean(pointIndexes=feature_point_indexes)
shape_pcaBasis_features = baselModel.getShapePCABasis(pointIndexes=feature_point_indexes)
shape_pcaVariance = baselModel.getShapePCAVariance()
        
triangles = baselModel.getModelMeshTriangles()
    
initializer = EstimationRefinementInitializer(numIterations=1)
optimizer = GradientDescent(
        mean=shape_mean_features,
        pcaBasis=shape_pcaBasis_features,
        pcaVariance=shape_pcaVariance,
        numIterations=1000,
        initializer=initializer)


inputHandler = SingleImageInput('./test_images/test_image_2.jpeg',
        './shape_predictor_68_face_landmarks.dat', dlib_points)
outputHandler = FaceModelMeshViewer(baselModel.getShapeMean(),
        baselModel.getShapePCABasis(), triangles)
optimizer.optimize(inputHandler, outputHandler)
            